# Exercise #2 - Create Fact & Dim Tables

Now that the three years of orders are combined into a single dataset, we can begin the processes of transforming the data.

In the one record, there are actually four sub-datasets:
* The order itself which is the aggregator of the other three datasets.
* The line items of each order which includes the price and quantity of each specific item.
* The sales rep placing the order.
* The customer placing the order - for the sake of simplicity, we will **not** break this dataset out and leave it as part of the order.

What we want to do next, is to extract all that data into their respective datasets (except the customer data). 

In other words, we want to normalize the data, in this case, to reduce data duplication.

This exercise is broken up into 5 steps:
* Exercise 2.A - Create & Use Database
* Exercise 2.B - Load & Cache Batch Orders
* Exercise 2.C - Extract Sales Reps
* Exercise 2.D - Extract Orders
* Exercise 2.E - Extract Line Items

In [5]:
user_db = "learning_db"
batch_source_path = "output/batch_orders_dirty.delta"
orders_table = "orders"
line_items_table = "line_items"
sales_reps_table = "sales_reps"
batch_temp_view = "batched_orders"

In [1]:
from util import get_spark_session

spark = get_spark_session()

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #2.A - Create &amp; Use Database</h2>

**In this step you will need to:**
* Create the database identified by the variable **`user_db`**
* Use the database identified by the variable **`user_db`** so that any tables created in this notebook are **NOT** added to the **`default`** database

**Special Notes**
* Do not hard-code the database name - in some scenarios this will result in validation errors.
* For assistence with the SQL command to create a database, see <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-syntax-ddl-create-database.html" target="_blank">CREATE DATABASE</a> on the Databricks docs website.
* For assistence with the SQL command to use a database, see <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-syntax-ddl-usedb.html" target="_blank">USE DATABASE</a> on the Databricks docs website.

### Implement Exercise #3.A

Implement your solution in the following cell:

In [ ]:
# TODO
# Use this cell to complete your solution

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #2.B - Load &amp; Cache Batch Orders</h2>

Next, we need to load the batch orders from the previous exercise and then cache them in preparation to transform the data later in this exercise.

**In this step you will need to:**
* Load the delta dataset we created in the previous exercise, identified by the variable **`batch_source_path`**.
* Using that same dataset, create a temporary view identified by the variable **`batch_temp_view`**.
* Cache the temporary view.

### Implement Exercise #3.B

Implement your solution in the following cell:

In [ ]:
# TODO
# Use this cell to complete your solution

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #2.C - Extract Sales Reps</h2>

Our batched orders from Exercise #1 contains thousands of orders and with every order, is the name, SSN, address and other information on the sales rep making the order.

We can use this data to create a table of just our sales reps.

If you consider that we have only ~100 sales reps, but thousands of orders, we are going to have a lot of duplicate data in this space.

Also unique to this set of data, is the fact that social security numbers were not always sanitized meaning sometime they were formatted with hyphens and in other cases they were not - this is something we will have to address here.

**In this step you will need to:**
* Load the table **`batched_orders`** (identified by the variable **`batch_temp_view`**)
* The SSN numbers have errors in them that we want to track - add the **`boolean`** column **`_error_ssn_format`** - for any case where **`sales_rep_ssn`** has a hypen in it, set this value to **`true`** otherwise **`false`**
* Convert various columns from their string representation to the specified type:
  * The column **`sales_rep_ssn`** should be represented as a **`Long`** (Note: You will have to first clean the column by removing extreneous hyphens in some records)
  * The column **`sales_rep_zip`** should be represented as an **`Integer`**
* Remove the columns not directly related to the sales-rep record:
  * Unrelated ID columns: **`submitted_at`**, **`order_id`**, **`customer_id`**
  * Shipping address columns: **`shipping_address_attention`**, **`shipping_address_address`**, **`shipping_address_city`**, **`shipping_address_state`**, **`shipping_address_zip`**
  * Product columns: **`product_id`**, **`product_quantity`**, **`product_sold_price`**
* Because there is one record per product ordered (many products per order), not to mention one sales rep placing many orders (many orders per sales rep), there will be duplicate records for our sales reps. Remove all duplicate records, making sure to exclude **`ingest_file_name`** and **`ingested_at`** from the evaluation of duplicate records
* Load the dataset to the managed delta table **`sales_rep_scd`** (identified by the variable **`sales_reps_table`**)

**Additional Requirements:**<br/>
The schema for the **`sales_rep_scd`** table must be:
* **`sales_rep_id`**:**`string`**
* **`sales_rep_ssn`**:**`long`**
* **`sales_rep_first_name`**:**`string`**
* **`sales_rep_last_name`**:**`string`**
* **`sales_rep_address`**:**`string`**
* **`sales_rep_city`**:**`string`**
* **`sales_rep_state`**:**`string`**
* **`sales_rep_zip`**:**`integer`**
* **`ingest_file_name`**:**`string`**
* **`ingested_at`**:**`timestamp`**
* **`_error_ssn_format`**:**`boolean`**

### Implement Exercise #2.C

Implement your solution in the following cell:

In [ ]:
# TODO
# Use this cell to complete your solution

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #2.D - Extract Orders</h2>

Our batched orders from Exercise 01 contains one line per product meaning there are multiple records per order.

The goal of this step is to extract just the order details (excluding the sales rep and line items)

**In this step you will need to:**
* Load the table **`batched_orders`** (identified by the variable **`batch_temp_view`**)
* Convert various columns from their string representation to the specified type:
  * The column **`submitted_at`** is a "unix epoch" (number of seconds since 1970-01-01 00:00:00 UTC) and should be represented as a **`Timestamp`**
  * The column **`shipping_address_zip`** should be represented as an **`Integer`**
* Remove the columns not directly related to the order record:
  * Sales reps columns: **`sales_rep_ssn`**, **`sales_rep_first_name`**, **`sales_rep_last_name`**, **`sales_rep_address`**, **`sales_rep_city`**, **`sales_rep_state`**, **`sales_rep_zip`**
  * Product columns: **`product_id`**, **`product_quantity`**, **`product_sold_price`**
* Because there is one record per product ordered (many products per order), there will be duplicate records for each order. Remove all duplicate records, making sure to exclude **`ingest_file_name`** and **`ingested_at`** from the evaluation of duplicate records
* Add the column **`submitted_yyyy_mm`** which is a **`string`** derived from **`submitted_at`** and is formatted as "**yyyy-MM**".
* Load the dataset to the managed delta table **`orders`** (identified by the variable **`orders_table`**)
  * In thise case, the data must also be partitioned by **`submitted_yyyy_mm`**

**Additional Requirements:**
* The schema for the **`orders`** table must be:
  * **`submitted_at:timestamp`**
  * **`submitted_yyyy_mm`** using the format "**yyyy-MM**"
  * **`order_id:string`**
  * **`customer_id:string`**
  * **`sales_rep_id:string`**
  * **`shipping_address_attention:string`**
  * **`shipping_address_address:string`**
  * **`shipping_address_city:string`**
  * **`shipping_address_state:string`**
  * **`shipping_address_zip:integer`**
  * **`ingest_file_name:string`**
  * **`ingested_at:timestamp`**

### Implement Exercise #2.D

Implement your solution in the following cell:

In [ ]:
# TODO
# Use this cell to complete your solution

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #2.E - Extract Line Items</h2>

Now that we have extracted sales reps and orders, we next want to extract the specific line items of each order.

**In this step you will need to:**
* Load the table **`batched_orders`** (identified by the variable **`batch_temp_view`**)
* Retain the following columns (see schema below)
  * The correlating ID columns: **`order_id`** and **`product_id`**
  * The two product-specific columns: **`product_quantity`** and **`product_sold_price`**
  * The two ingest columns: **`ingest_file_name`** and **`ingested_at`**
* Convert various columns from their string representation to the specified type:
  * The column **`product_quantity`** should be represented as an **`Integer`**
  * The column **`product_sold_price`** should be represented as an **`Decimal`** with two decimal places as in **`decimal(10,2)`**
* Load the dataset to the managed delta table **`line_items`** (identified by the variable **`line_items_table`**)

**Additional Requirements:**
* The schema for the **`line_items`** table must be:
  * **`order_id`**:**`string`**
  * **`product_id`**:**`string`**
  * **`product_quantity`**:**`integer`**
  * **`product_sold_price`**:**`decimal(10,2)`**
  * **`ingest_file_name`**:**`string`**
  * **`ingested_at`**:**`timestamp`**

### Implement Exercise #2.E

Implement your solution in the following cell:

In [ ]:
# TODO
# Use this cell to complete your solution